# COMPLETE ADVANCED FACE DETECTION SYSTEM

## Import Libraries


In [1]:
import cv2
import numpy as np
import mediapipe as mp
import math
import time
import os
from collections import deque

## MODULE 1: face_detector
Core face detection using MediaPipe

In [2]:
class MediaPipeFaceDetector:
    def __init__(self):
        self.mp_face_detection = mp.solutions.face_detection
        self.face_detection = self.mp_face_detection.FaceDetection(
            model_selection=0,
            min_detection_confidence=0.5
        )
    
    def detect_faces(self, frame):
        """Detect faces using MediaPipe"""
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.face_detection.process(rgb_frame)
        
        faces = []
        if results.detections:
            for detection in results.detections:
                bbox = detection.location_data.relative_bounding_box
                h, w, _ = frame.shape
                
                x = max(0, int(bbox.xmin * w))
                y = max(0, int(bbox.ymin * h))
                width = min(w - x, int(bbox.width * w))
                height = min(h - y, int(bbox.height * h))
                
                confidence = detection.score[0]
                
                faces.append({
                    'bbox': (x, y, width, height),
                    'confidence': confidence
                })
        
        return faces

## MODULE 2: landmarks_detector.py

468 facial landmarks detection

In [3]:
class FacialLandmarksDetector:
    def __init__(self):
        self.mp_face_mesh = mp.solutions.face_mesh
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_drawing_styles = mp.solutions.drawing_styles
        
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            static_image_mode=False,
            max_num_faces=5,
            refine_landmarks=True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
    
    def get_landmarks(self, frame):
        """Get 468 facial landmarks"""
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.face_mesh.process(rgb_frame)
        
        landmarks_data = []
        if results.multi_face_landmarks:
            h, w, _ = frame.shape
            for face_landmarks in results.multi_face_landmarks:
                landmarks = []
                for landmark in face_landmarks.landmark:
                    x = int(landmark.x * w)
                    y = int(landmark.y * h)
                    landmarks.append((x, y))
                landmarks_data.append(landmarks)
        
        return landmarks_data, results
    
    def draw_landmarks(self, frame, landmarks):
        """Draw landmarks on frame"""
        for point in landmarks:
            cv2.circle(frame, point, 1, (0, 255, 0), -1)
    
    def draw_face_mesh(self, frame, face_mesh_results):
        """Draw 3D face mesh"""
        if face_mesh_results.multi_face_landmarks:
            for face_landmarks in face_mesh_results.multi_face_landmarks:
                self.mp_drawing.draw_landmarks(
                    frame, face_landmarks,
                    self.mp_face_mesh.FACEMESH_CONTOURS,
                    None,
                    self.mp_drawing_styles.get_default_face_mesh_contours_style()
                )

## MODULE 3: age_gender_detector.py

 Age and gender prediction

In [4]:
class AgeGenderDetector:
    def __init__(self):
        self.age_net = None
        self.gender_net = None
        self.age_list = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
        self.gender_list = ['Male', 'Female']
        self.load_models()
    
    def load_models(self):
        """Load age and gender models"""
        try:
            if os.path.exists('models/age_net.caffemodel') and os.path.exists('models/age_deploy.prototxt'):
                self.age_net = cv2.dnn.readNet('models/age_net.caffemodel', 'models/age_deploy.prototxt')
                print("✅ Age model loaded")
            
            if os.path.exists('models/gender_net.caffemodel') and os.path.exists('models/gender_deploy.prototxt'):
                self.gender_net = cv2.dnn.readNet('models/gender_net.caffemodel', 'models/gender_deploy.prototxt')
                print("✅ Gender model loaded")
                
        except Exception as e:
            print(f"❌ Error loading models: {e}")
    
    def predict(self, face_roi):
        """Predict age and gender"""
        if self.age_net is None or self.gender_net is None:
            return "Model Error", "Model Error", 0.0
        
        if face_roi.size == 0:
            return "Invalid", "Invalid", 0.0
        
        try:
            blob = cv2.dnn.blobFromImage(
                face_roi, 1.0, (227, 227),
                (78.4263377603, 87.7689143744, 114.895847746)
            )
            
            # Age prediction
            self.age_net.setInput(blob)
            age_predictions = self.age_net.forward()
            predicted_age = self.age_list[age_predictions[0].argmax()]
            age_confidence = age_predictions[0].max()
            
            # Gender prediction
            self.gender_net.setInput(blob)
            gender_predictions = self.gender_net.forward()
            predicted_gender = self.gender_list[gender_predictions[0].argmax()]
            gender_confidence = gender_predictions[0].max()
            
            combined_confidence = (age_confidence + gender_confidence) / 2
            return predicted_age, predicted_gender, combined_confidence
            
        except Exception as e:
            return "Error", "Error", 0.0

## MODULE 4: ar_filters.py
Augmented Reality filters

In [5]:
class ARFilters:
    def __init__(self):
        self.create_filters()
    
    def create_filters(self):
        """Create AR filter assets"""
        # Sunglasses
        self.sunglasses = np.zeros((50, 150, 4), dtype=np.uint8)
        cv2.rectangle(self.sunglasses, (0, 0), (149, 49), (0, 0, 0, 200), -1)
        cv2.rectangle(self.sunglasses, (0, 0), (149, 49), (255, 255, 255, 255), 3)
        cv2.line(self.sunglasses, (75, 20), (75, 30), (255, 255, 255, 255), 4)
        
        # Mustache
        self.mustache = np.zeros((40, 100, 4), dtype=np.uint8)
        cv2.ellipse(self.mustache, (50, 25), (45, 15), 0, 0, 180, (101, 67, 33, 255), -1)
        
        # Hat
        self.hat = np.zeros((80, 120, 4), dtype=np.uint8)
        cv2.rectangle(self.hat, (10, 60), (110, 75), (255, 0, 0, 255), -1)
        cv2.rectangle(self.hat, (25, 10), (95, 65), (139, 0, 0, 255), -1)
    
    def apply_sunglasses(self, frame, landmarks):
        """Apply sunglasses filter"""
        if len(landmarks) < 468:
            return
        
        left_eye = landmarks[33]
        right_eye = landmarks[362]
        
        eye_distance = math.sqrt((right_eye[0] - left_eye[0])**2 + (right_eye[1] - left_eye[1])**2)
        center_x = (left_eye[0] + right_eye[0]) // 2
        center_y = (left_eye[1] + right_eye[1]) // 2
        
        scale_factor = eye_distance / 100
        new_width = int(self.sunglasses.shape[1] * scale_factor)
        new_height = int(self.sunglasses.shape[0] * scale_factor)
        
        if new_width > 10 and new_height > 10:
            resized = cv2.resize(self.sunglasses, (new_width, new_height))
            start_x = center_x - new_width // 2
            start_y = center_y - new_height // 2
            self.overlay_alpha(frame, resized, start_x, start_y)
    
    def apply_mustache(self, frame, landmarks):
        """Apply mustache filter"""
        if len(landmarks) < 468:
            return
        
        nose_bottom = landmarks[2]
        mouth_top = landmarks[12]
        
        center_x = (nose_bottom[0] + mouth_top[0]) // 2
        center_y = (nose_bottom[1] + mouth_top[1]) // 2
        
        face_width = abs(landmarks[454][0] - landmarks[234][0])
        scale_factor = face_width / 200
        
        new_width = int(self.mustache.shape[1] * scale_factor)
        new_height = int(self.mustache.shape[0] * scale_factor)
        
        if new_width > 10 and new_height > 10:
            resized = cv2.resize(self.mustache, (new_width, new_height))
            start_x = center_x - new_width // 2
            start_y = center_y - new_height // 2
            self.overlay_alpha(frame, resized, start_x, start_y)
    
    def apply_hat(self, frame, landmarks):
        """Apply hat filter"""
        if len(landmarks) < 468:
            return
        
        forehead_center = landmarks[10]
        face_width = abs(landmarks[454][0] - landmarks[234][0])
        scale_factor = face_width / 150
        
        new_width = int(self.hat.shape[1] * scale_factor)
        new_height = int(self.hat.shape[0] * scale_factor)
        
        if new_width > 10 and new_height > 10:
            resized = cv2.resize(self.hat, (new_width, new_height))
            start_x = forehead_center[0] - new_width // 2
            start_y = forehead_center[1] - new_height
            self.overlay_alpha(frame, resized, start_x, start_y)
    
    def overlay_alpha(self, background, overlay, x, y):
        """Overlay with alpha blending"""
        if overlay.shape[2] != 4:
            return
        
        h, w = overlay.shape[:2]
        if x < 0 or y < 0 or x + w > background.shape[1] or y + h > background.shape[0]:
            return
        
        alpha = overlay[:, :, 3] / 255.0
        for c in range(3):
            background[y:y+h, x:x+w, c] = (
                alpha * overlay[:, :, c] + 
                (1 - alpha) * background[y:y+h, x:x+w, c]
            )


## MODULE 5: ui_manager.py
User interface and controls

In [6]:
class UIManager:
    def __init__(self):
        self.show_controls = True
        self.show_stats = True
        self.show_performance = True
        
        # Color scheme (BGR format for OpenCV)
        self.colors = {
            'primary': (50, 205, 50),      # Lime green
            'secondary': (255, 215, 0),    # Gold
            'accent': (255, 69, 0),        # Red orange
            'background': (40, 40, 40),    # Dark gray
            'text_primary': (255, 255, 255), # White
            'text_secondary': (200, 200, 200), # Light gray
            'overlay': (0, 0, 0),          # Black
            'success': (0, 255, 0),        # Green
            'warning': (0, 255, 255),      # Yellow
            'error': (0, 0, 255)           # Red
        }
        
        # UI layout settings
        self.margin = 15
        self.padding = 10
        self.corner_radius = 15
        
    def draw_rounded_rectangle(self, frame, pt1, pt2, color, thickness=-1, radius=15):
        """Draw a rounded rectangle"""
        x1, y1 = pt1
        x2, y2 = pt2
        
        # Create overlay for rounded rectangle
        overlay = np.zeros_like(frame)
        
        # Main rectangle
        cv2.rectangle(overlay, (x1 + radius, y1), (x2 - radius, y2), color, thickness)
        cv2.rectangle(overlay, (x1, y1 + radius), (x2, y2 - radius), color, thickness)
        
        # Corners
        cv2.circle(overlay, (x1 + radius, y1 + radius), radius, color, thickness)
        cv2.circle(overlay, (x2 - radius, y1 + radius), radius, color, thickness)
        cv2.circle(overlay, (x1 + radius, y2 - radius), radius, color, thickness)
        cv2.circle(overlay, (x2 - radius, y2 - radius), radius, color, thickness)
        
        # Blend with original frame
        if thickness == -1:  # Filled rectangle
            frame[:] = cv2.addWeighted(frame, 0.7, overlay, 0.3, 0)
        else:  # Border only
            mask = cv2.cvtColor(overlay, cv2.COLOR_BGR2GRAY)
            frame[mask > 0] = overlay[mask > 0]
    
    def draw_modern_controls(self, frame):
        """Draw modern, sleek control panel"""
        if not self.show_controls:
            return
        
        height, width = frame.shape[:2]
        
        # Control panel dimensions
        panel_width = 320
        panel_height = 200
        panel_x = self.margin
        panel_y = height - panel_height - self.margin
        
        # Draw main panel background
        self.draw_rounded_rectangle(
            frame, 
            (panel_x, panel_y), 
            (panel_x + panel_width, panel_y + panel_height),
            self.colors['overlay'],
            -1,
            self.corner_radius
        )
        
        # Panel border
        self.draw_rounded_rectangle(
            frame,
            (panel_x, panel_y),
            (panel_x + panel_width, panel_y + panel_height),
            self.colors['primary'],
            2,
            self.corner_radius
        )
        
        # Title bar
        title_height = 35
        self.draw_rounded_rectangle(
            frame,
            (panel_x + 5, panel_y + 5),
            (panel_x + panel_width - 5, panel_y + title_height),
            self.colors['primary'],
            -1,
            10
        )
        
        # Title text
        cv2.putText(frame, "CONTROLS", 
                   (panel_x + 20, panel_y + 25), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, 
                   self.colors['text_primary'], 2)
        
        # Control categories
        filter_controls = [
            ("S", "Sunglasses Filter"),
            ("M", "Mustache Filter"),
            ("H", "Hat Filter")
        ]
        
        view_controls = [
            ("L", "Toggle Landmarks"),
            ("F", "Toggle Face Mesh"),
            ("A", "Age/Gender Detection")
        ]
        
        system_controls = [
            ("C", "Hide/Show Controls"),
            ("Q", "Quit Application")
        ]
        
        # Draw filter controls
        y_offset = panel_y + 50
        cv2.putText(frame, "FILTERS:", 
                   (panel_x + 15, y_offset), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, 
                   self.colors['secondary'], 1)
        
        for i, (key, desc) in enumerate(filter_controls):
            y_pos = y_offset + 15 + (i * 18)
            # Key background
            cv2.rectangle(frame, 
                         (panel_x + 25, y_pos - 12), 
                         (panel_x + 40, y_pos + 2), 
                         self.colors['accent'], -1)
            cv2.putText(frame, key, 
                       (panel_x + 28, y_pos - 2), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, 
                       self.colors['text_primary'], 1)
            cv2.putText(frame, desc, 
                       (panel_x + 50, y_pos - 2), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.35, 
                       self.colors['text_secondary'], 1)
        
        # Draw view controls
        y_offset = panel_y + 110
        cv2.putText(frame, "VIEW OPTIONS:", 
                   (panel_x + 15, y_offset), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, 
                   self.colors['secondary'], 1)
        
        for i, (key, desc) in enumerate(view_controls):
            y_pos = y_offset + 15 + (i * 18)
            cv2.rectangle(frame, 
                         (panel_x + 25, y_pos - 12), 
                         (panel_x + 40, y_pos + 2), 
                         self.colors['accent'], -1)
            cv2.putText(frame, key, 
                       (panel_x + 28, y_pos - 2), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, 
                       self.colors['text_primary'], 1)
            cv2.putText(frame, desc, 
                       (panel_x + 50, y_pos - 2), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.35, 
                       self.colors['text_secondary'], 1)
        
        # Draw system controls
        y_offset = panel_y + 170
        for i, (key, desc) in enumerate(system_controls):
            x_pos = panel_x + 25 + (i * 140)
            cv2.rectangle(frame, 
                         (x_pos, y_offset - 12), 
                         (x_pos + 15, y_offset + 2), 
                         self.colors['warning'] if key == 'Q' else self.colors['accent'], -1)
            cv2.putText(frame, key, 
                       (x_pos + 3, y_offset - 2), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, 
                       self.colors['text_primary'], 1)
            cv2.putText(frame, desc.split()[0], 
                       (x_pos + 20, y_offset - 2), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.35, 
                       self.colors['text_secondary'], 1)
    
    def draw_status_panel(self, frame, current_filter, fps, face_count, confidence_avg=0):
        """Draw professional status panel"""
        if not self.show_stats:
            return
        
        height, width = frame.shape[:2]
        
        # Status panel dimensions
        panel_width = 280
        panel_height = 150
        panel_x = width - panel_width - self.margin
        panel_y = self.margin
        
        # Draw status panel background
        self.draw_rounded_rectangle(
            frame,
            (panel_x, panel_y),
            (panel_x + panel_width, panel_y + panel_height),
            self.colors['overlay'],
            -1,
            self.corner_radius
        )
        
        # Panel border with color based on performance
        border_color = self.colors['success'] if fps > 25 else self.colors['warning'] if fps > 15 else self.colors['error']
        self.draw_rounded_rectangle(
            frame,
            (panel_x, panel_y),
            (panel_x + panel_width, panel_y + panel_height),
            border_color,
            2,
            self.corner_radius
        )
        
        # Title
        cv2.putText(frame, "SYSTEM STATUS", 
                   (panel_x + 15, panel_y + 25), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                   self.colors['primary'], 2)
        
        # Status indicators
        status_items = [
            ("Active Filter:", current_filter.upper() if current_filter != "none" else "NONE"),
            ("Faces Detected:", str(face_count)),
            ("Performance:", f"{fps:.1f} FPS"),
            ("Detection Quality:", f"{confidence_avg:.1%}" if confidence_avg > 0 else "N/A")
        ]
        
        for i, (label, value) in enumerate(status_items):
            y_pos = panel_y + 55 + (i * 22)
            
            # Label
            cv2.putText(frame, label, 
                       (panel_x + 15, y_pos), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, 
                       self.colors['text_secondary'], 1)
            
            # Value with appropriate color
            value_color = self.colors['primary']
            if label == "Performance:":
                value_color = self.colors['success'] if fps > 25 else self.colors['warning'] if fps > 15 else self.colors['error']
            elif label == "Active Filter:" and value != "NONE":
                value_color = self.colors['accent']
            
            cv2.putText(frame, value, 
                       (panel_x + 140, y_pos), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.45, 
                       value_color, 1)
    
    def draw_performance_graph(self, frame, fps_history):
        """Draw mini performance graph"""
        if not self.show_performance or len(fps_history) < 2:
            return
        
        height, width = frame.shape[:2]
        
        # Graph dimensions
        graph_width = 200
        graph_height = 60
        graph_x = width - graph_width - self.margin
        graph_y = height - graph_height - self.margin - 40
        
        # Background
        cv2.rectangle(frame, 
                        (graph_x, graph_y), 
                        (graph_x + graph_width, graph_y + graph_height),
                        self.colors['overlay'], -1)
        
        # Border
        cv2.rectangle(frame, 
                        (graph_x, graph_y), 
                        (graph_x + graph_width, graph_y + graph_height),
                        self.colors['primary'], 1)
        
        # Title
        cv2.putText(frame, "FPS GRAPH", 
                        (graph_x + 5, graph_y - 5), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.3, 
                        self.colors['text_secondary'], 1)
        
        # Draw FPS line
        if len(fps_history) > 1:
            # Convert deque to list and get recent values
            fps_list = list(fps_history)
            recent_fps = fps_list[-min(50, len(fps_list)):]  # Last 50 frames or all available
            
            max_fps = max(recent_fps) if max(recent_fps) > 30 else 30
            points = []
            
            for i, fps in enumerate(recent_fps):
                x = graph_x + int((i / len(recent_fps)) * graph_width)
                y = graph_y + graph_height - int((fps / max_fps) * graph_height)
                points.append((x, y))
            
            # Draw line
            for i in range(1, len(points)):
                color = self.colors['success'] if recent_fps[i] > 25 else self.colors['warning']
                cv2.line(frame, points[i-1], points[i], color, 2)
    
    def draw_notification(self, frame, message, notification_type="info"):
        """Draw temporary notification"""
        height, width = frame.shape[:2]
        
        # Notification colors
        colors_map = {
            "info": self.colors['primary'],
            "success": self.colors['success'],
            "warning": self.colors['warning'],
            "error": self.colors['error']
        }
        
        color = colors_map.get(notification_type, self.colors['primary'])
        
        # Notification dimensions
        text_size = cv2.getTextSize(message, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
        notif_width = text_size[0] + 40
        notif_height = 40
        notif_x = (width - notif_width) // 2
        notif_y = 50
        
        # Background
        self.draw_rounded_rectangle(
            frame,
            (notif_x, notif_y),
            (notif_x + notif_width, notif_y + notif_height),
            color,
            -1,
            10
        )
        
        # Text
        cv2.putText(frame, message, 
                    (notif_x + 20, notif_y + 25), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, 
                    self.colors['text_primary'], 2)
    
    def toggle_controls(self):
        """Toggle control panel visibility"""
        self.show_controls = not self.show_controls
    
    def toggle_stats(self):
        """Toggle statistics panel visibility"""
        self.show_stats = not self.show_stats
    
    def toggle_performance(self):
        """Toggle performance graph visibility"""
        self.show_performance = not self.show_performance

## MODULE 6: main_app.py
Main application that combines everything

In [7]:
class FaceDetectionApp:
    def __init__(self):
        # Initialize all components
        self.face_detector = MediaPipeFaceDetector()
        self.landmarks_detector = FacialLandmarksDetector()
        self.age_gender_detector = AgeGenderDetector()
        self.ar_filters = ARFilters()
        self.ui_manager = UIManager()
        
        # Settings
        self.current_filter = "none"
        self.show_landmarks = False
        self.show_age_gender = False
        self.show_face_mesh = False
        
        # Performance tracking
        self.fps_counter = deque(maxlen=30)
    
    def process_frame(self, frame):
        """Process single frame"""
        frame_start = time.time()
        
        # Detect faces
        faces = self.face_detector.detect_faces(frame)
        
        # Get landmarks
        landmarks_data, face_mesh_results = self.landmarks_detector.get_landmarks(frame)
        
        # Calculate average confidence
        avg_confidence = 0
        if faces:
            avg_confidence = sum(face['confidence'] for face in faces) / len(faces)
        
        # Draw face boxes and age/gender
        for face in faces:
            x, y, w, h = face['bbox']
            confidence = face['confidence']
            
            color = (0, 255, 0) if confidence > 0.7 else (0, 255, 255)
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, f'{confidence:.2f}', (x, y-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
            
            if self.show_age_gender:
                face_roi = frame[y:y+h, x:x+w]
                if face_roi.size > 0:
                    age, gender, _ = self.age_gender_detector.predict(face_roi)
                    cv2.putText(frame, f'{gender}, {age}', (x, y-30), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        # Apply filters and landmarks
        if landmarks_data:
            for landmarks in landmarks_data:
                if self.show_landmarks:
                    self.landmarks_detector.draw_landmarks(frame, landmarks)
                
                if self.current_filter == "sunglasses":
                    self.ar_filters.apply_sunglasses(frame, landmarks)
                elif self.current_filter == "mustache":
                    self.ar_filters.apply_mustache(frame, landmarks)
                elif self.current_filter == "hat":
                    self.ar_filters.apply_hat(frame, landmarks)
        
        if self.show_face_mesh:
            self.landmarks_detector.draw_face_mesh(frame, face_mesh_results)
        
        # Update FPS
        frame_time = time.time() - frame_start
        if frame_time > 0:
            self.fps_counter.append(1.0 / frame_time)
        
        # Draw UI with new methods
        fps = np.mean(self.fps_counter) if self.fps_counter else 0
        self.ui_manager.draw_status_panel(frame, self.current_filter, fps, len(faces), avg_confidence)
        self.ui_manager.draw_modern_controls(frame)
        self.ui_manager.draw_performance_graph(frame, self.fps_counter)
        
        return frame
    
    def handle_key(self, key):
        """Handle keyboard input"""
        if key == ord('s'):
            self.current_filter = "sunglasses" if self.current_filter != "sunglasses" else "none"
        elif key == ord('m'):
            self.current_filter = "mustache" if self.current_filter != "mustache" else "none"
        elif key == ord('h'):
            self.current_filter = "hat" if self.current_filter != "hat" else "none"
        elif key == ord('l'):
            self.show_landmarks = not self.show_landmarks
        elif key == ord('f'):
            self.show_face_mesh = not self.show_face_mesh
        elif key == ord('a'):
            self.show_age_gender = not self.show_age_gender
        elif key == ord('c'):
            self.ui_manager.toggle_controls()
    
    def run(self):
        """Main application loop"""
        cap = cv2.VideoCapture(0)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        
        print("🚀 Face Detection System Started!")
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame = self.process_frame(frame)
            cv2.imshow('Face Detection System', frame)
            
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            else:
                self.handle_key(key)
        
        cap.release()
        cv2.destroyAllWindows()

## MODULE 7: run.py
Simple runner script

In [ ]:

if __name__ == "__main__":
    app = FaceDetectionApp()
    app.run()

"""
📁 SUGGESTED FILE STRUCTURE:

face_detection_project/
├── models/
│   ├── age_net.caffemodel
│   ├── age_deploy.prototxt
│   ├── gender_net.caffemodel
│   └── gender_deploy.prototxt
├── face_detector.py          # Module 1
├── landmarks_detector.py     # Module 2  
├── age_gender_detector.py    # Module 3
├── ar_filters.py            # Module 4
├── ui_manager.py            # Module 5
├── main_app.py              # Module 6
└── run.py                   # Module 7

🚀 TO RUN:
python run.py

💡 BENEFITS:
- Clean, organized code
- Easy to maintain and modify
- Each module has a single responsibility
- Can test individual components
- Easy to add new features
"""